# Möbius-Digit-Thread Explorer (v16)
Restores the *v6* sweet-spot and runs a precise sweep:
- φ-micro centred at **0.003944 ± 5×10⁻⁷** (11 points)
- One Möbius breath plus optional damped power λ ∈ {0, 0.1, 0.2, 0.3, 0.4}
- tiny = α / 10⁴, seeds 64 and 128 Fibonacci fractions

The table lists the 20 best rows sorted by relative error |x - α| / α and
flags any miss < 1 %.  A sub-0.1 % miss prints a breakthrough banner.

In [3]:
import mpmath as mp, itertools, pandas as pd
mp.mp.dps = 260  # high precision

# --- CODATA constants -------------------------------------------------
ALPHA_INV = mp.mpf('137.035999084')  # 2022 value 1/α
ALPHA     = 1 / ALPHA_INV
PHI       = (1 + mp.sqrt(5)) / 2

# --- Fibonacci helpers -----------------------------------------------
def fib_word_bits(n: int) -> str:
    a, b = '0', '01'
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

def fib_fraction(bits: str) -> mp.mpf:
    s = mp.mpf(0)
    for i, bit in enumerate(bits):
        if bit == '1':
            s += mp.power(2, -(i + 1))
    return s

F64  = fib_fraction(fib_word_bits(64))
F128 = fib_fraction(fib_word_bits(128))

# --- Möbius maps ------------------------------------------------------
def breath_once(x: mp.mpf, tiny: mp.mpf, phi: mp.mpf) -> mp.mpf:
    # 1) φ-scale reciprocal; 2) inject α; 3) Möbius neutraliser 1/(9x); 4) reciprocal
    x = phi * x / tiny
    x = x + tiny
    x = 1 / x / 9
    x = 1 / x
    return x

def damped_breath(x: mp.mpf, tiny: mp.mpf, phi: mp.mpf, lam: float) -> mp.mpf:
    """Apply one Möbius breath, then real-safe damping
    x -> sign(x) * |x|**lam  with lam in [0, 0.4].
    lam = 0 gives no damping."""
    x2 = breath_once(x, tiny, phi)
    return mp.power(abs(x2), lam) * mp.sign(x2)

# --- sweep parameters --------------------------------------------------
phi_center = mp.mpf('0.003944')
phi_vals   = [phi_center + i * 5e-7 for i in range(-5, 6)]
k_vals     = [3.8, 4.0]
lam_vals   = [0.0, 0.1, 0.2, 0.3, 0.4]
tiny_exp   = 4  # tiny = α / 10**tiny_exp

rows = []
for bits in (64, 128):
    seed = F64 if bits == 64 else F128
    tiny = ALPHA / mp.power(10, tiny_exp)
    for k in k_vals:
        for phi in phi_vals:
            for lam in lam_vals:
                x = damped_breath(seed + tiny, tiny, phi, lam)
                rel_err = abs(x - ALPHA) / ALPHA
                rows.append(dict(bits=bits, k=k, phi=float(phi), lam=lam,
                                 rel_err=rel_err))

df = (pd.DataFrame(rows)
        .sort_values('rel_err')
        .reset_index(drop=True))

print("Top 20 rows by relative error:")
display(df.head(20))

best = df.iloc[0]
if best.rel_err < 0.01:
    print(f"*** Hit <1% with bits={best.bits}, k={best.k}, "
          f"phi={best.phi}, lam={best.lam}, rel={best.rel_err:.4e} ***")
if best.rel_err < 0.001:
    print("### Sub-0.1% breakthrough! ###")

Top 20 rows by relative error:


,bits,k,phi,lam,rel_err
0,64,3.8,0.003941,0.0,136.035999084
1,128,3.8,0.003942,0.0,136.035999084
2,128,3.8,0.003941,0.0,136.035999084
3,64,4.0,0.003947,0.0,136.035999084
4,64,4.0,0.003946,0.0,136.035999084
5,64,4.0,0.003946,0.0,136.035999084
6,64,4.0,0.003945,0.0,136.035999084
7,128,3.8,0.003942,0.0,136.035999084
8,64,4.0,0.003945,0.0,136.035999084
9,64,4.0,0.003943,0.0,136.035999084
